In [ ]:
#pip install transformers

In [ ]:
#pip install spicy

In [ ]:
#pip install torch

In [4]:
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import pandas as pd 
import torch


In [5]:
#using model from https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest, referred to it at thesis
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
pip install openpyxl

In [13]:
df = pd.read_excel(r'read in your file')

In [15]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

In [16]:
#chaging some data types
df['author_id'] = df['author_id'].astype(int)

In [17]:
df['tweet_id'] = df['tweet_id'].astype(int)

In [18]:
df['text'] = df['text'].astype(str)

In [19]:
df['text_check'] = df['text'].apply(eval).apply(' '.join)

In [21]:
df['positive'] = 0.0
df['negative'] = 0.0
df['neutral'] = 0.0
df['sentiment_label'] = ""
df['sentiment_score'] = 0.0

In [ ]:
for i, row in df.iterrows():
    encoded_data = tokenizer.encode_plus(row['text'],
                                          padding=True,
                                          truncation=True,
                                          max_length=300,
                                          return_tensors='pt')
    output = model(**encoded_data)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]

    sentiment_label = config.id2label[ranking[0]]
    sentiment_score = np.round(float(scores[ranking[0]]), 4)

    df.at[i, 'positive'] = scores[2]
    df.at[i, 'negative'] = scores[0]
    df.at[i, 'neutral'] = scores[1]
    df.at[i, 'sentiment_label'] = sentiment_label
    df.at[i, 'sentiment_score'] = sentiment_score

In [ ]:
df.head()